In [6]:
import urllib.request as ur # import para obtener la página web 
import re                   # import para utilizar expresiones regulares 

# Obtener página web con incidencias de Madrid 
#f = ur.urlopen('http://infocar.dgt.es/etraffic/Incidencias?ca=13&provIci=&caracter=acontecimiento&accion_consultar=Consultar&IncidenciasRETENCION=IncidenciasRETENCION&IncidenciasPUERTOS=IncidenciasPUERTOS&IncidenciasMETEOROLOGICA=IncidenciasMETEOROLOGICA&IncidenciasEVENTOS=IncidenciasEVENTOS&IncidenciasOTROS=IncidenciasOTROS&IncidenciasRESTRICCIONES=IncidenciasRESTRICCIONES&ordenacion=fechahora_ini-DESC')

f = ur.urlopen('http://infocar.dgt.es/etraffic/Incidencias?ca=&provIci=&caracter=acontecimiento&accion_consultar=Consultar&IncidenciasRETENCION=IncidenciasRETENCION&IncidenciasPUERTOS=IncidenciasPUERTOS&IncidenciasMETEOROLOGICA=IncidenciasMETEOROLOGICA&IncidenciasEVENTOS=IncidenciasEVENTOS&IncidenciasOTROS=IncidenciasOTROS&IncidenciasRESTRICCIONES=IncidenciasRESTRICCIONES&ordenacion=fechahora_ini-DESC')
s = f.read().decode()
f.close()


In [7]:
# Método para eliminar tags html que no nos interesan con ER 
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,'',raw_html)
    return cleantext

In [8]:
from pprint import pprint     # import para hacer prints boninitos 

dividedByTr = s.split("<tr") # Dividir el string por tr para incidencias con hora fin 
endTimeElements = [] # Almacenará elementos que tienen hora fin 

# Obtener las incidencias que tienen hora fin 
print("Incidencias con hora fin")
endTime = re.compile(".*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?tr>",re.DOTALL)
for x in dividedByTr:
    confin = endTime.findall(x)
    #confin = re.findall(".*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?tr>",x,re.DOTALL)
    if len(confin): # Si el elemento contiene algún dato se agrega a la lista de elementos con hora fin
        endTimeElements.append(confin[0]) 
        pprint(confin)

# Obtener todas las incidencias 
print("\nTodas las incidencias") # Las cuales incluyen las que tienen hora fin pero no incluyen ese dato
conjunto = re.findall("<tr.*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d)?.*?(\d\d/\d\d/\d\d\d\d)?.*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?",s,re.DOTALL)
pprint(conjunto)



Incidencias con hora fin
[('06:00',
  '07/03/2019',
  '23:00',
  '07/03/2019',
  'M-111',
  'RESTRICCIONES A MERCANCÍAS PELIGROSAS</b> por <b>CIRCULACIÓN</b> en<b> '
  'Túnel Barajas</b>: </span> <span style="margin-top:10px; float:left; '
  'clear:both">  -  La carretera  <b> <span style="color:#ab3000">M-111 '
  '</span> </b> a la altura de <b>MADRID (MADRID)</b> desde el <b> km 1.1</b> '
  'al <b> km 5</b> sentido <b>AMBOS SENTIDOS')]
[('13:00',
  '06/03/2019',
  '00:00',
  '10/03/2019',
  'N-124',
  'CARRIL(ES) CERRADO(S)</b> por <b>OBRAS</b> con circulación irregular  '
  'en<b></b>: </span> <span style="margin-top:10px; float:left; clear:both">  '
  '- La CARRETERA NACIONAL <b> <span style="color:#ab3000">N-124 </span> </b> '
  'a la altura de <b>HARO (RIOJA, LA)</b> desde el <b> km 41.5</b> al <b> km '
  '43.5</b> sentido <b>CRECIENTE DE LA KILOMETRACIÓN</b> hacia <b>BRIÑAS')]
[('15:48',
  '06/02/2019',
  '23:59',
  '31/12/2019',
  'N-340',
  'RESTRICCIONES A VEHÍCULOS PESADOS</

In [9]:
diccionario = {} # variable que se utilizará para almacenar las incidencias una sola vez 

for x in range(len(endTimeElements)): # se ejecuta para cada elemento con hora fin 
    identificador = endTimeElements[x][0] + endTimeElements[x][1] + endTimeElements[x][4]  # concatenación de valores para crear identificador para diccionario
    diccionario.update({identificador : endTimeElements[x]}) # agregación de valores al diccionario

for x in range(len(conjunto)): # se ejecuta para cada elemento
    identificador = conjunto[x][0] + conjunto[x][1] + conjunto[x][4] # concatenación de valores para crear identificador para diccionario
    if diccionario.get(identificador) == None:    # En caso de que el identificador exista, salta este paso (es decir, el elemento tenía hora fin)
        diccionario.update({identificador : conjunto[x]}) # Si el identificador no existe, se crea un nuevo elemento en el diccionario
'''
Para todas las carreteras
Indica que hubo un atasco si la incidencia tiene hora fin 
Indica que existe un atasco si la incidencia no tiene hora fin 
'''
        
for x in diccionario:
    valores = diccionario.get(x) # obtener el valor correspondiente para el identificador 
    fin = "" # declarar variable para mensaje en caso de que la incidencia tenga hora fin
    cleantext =  cleanhtml(valores[5]) # quitar las etiquetas de html del texto que describe la incidencia 
    if valores[2] != '':  # En caso de que exista una hora fin
        mensaje = "Hubo un atasco en la carretera " + valores[4]
        fin = " el cual terminó a las " + valores[2] + " el día " + valores[3]
    else: 
        mensaje = "Existe un atasco en la carretera " + valores[4]
    mensaje = mensaje + " que inicio a las " + valores[0] + " el día " + valores[1]
    mensaje = mensaje + fin
    mensaje = mensaje + " con la siguientes características:\n" +  cleantext
    
    print(mensaje+"\n")
    


Hubo un atasco en la carretera M-111 que inicio a las 06:00 el día 07/03/2019 el cual terminó a las 23:00 el día 07/03/2019 con la siguientes características:
RESTRICCIONES A MERCANCÍAS PELIGROSAS por CIRCULACIÓN en Túnel Barajas:    -  La carretera   M-111   a la altura de MADRID (MADRID) desde el  km 1.1 al  km 5 sentido AMBOS SENTIDOS

Hubo un atasco en la carretera N-124 que inicio a las 13:00 el día 06/03/2019 el cual terminó a las 00:00 el día 10/03/2019 con la siguientes características:
CARRIL(ES) CERRADO(S) por OBRAS con circulación irregular  en:    - La CARRETERA NACIONAL  N-124   a la altura de HARO (RIOJA, LA) desde el  km 41.5 al  km 43.5 sentido CRECIENTE DE LA KILOMETRACIÓN hacia BRIÑAS

Hubo un atasco en la carretera N-340 que inicio a las 15:48 el día 06/02/2019 el cual terminó a las 23:59 el día 31/12/2019 con la siguientes características:
RESTRICCIONES A VEHÍCULOS PESADOS por CIRCULACIÓN en Salida 278 de la A-7 (Nules) a Salida 997 a la AP-7 (Oropesa):    -  La car

In [10]:
'''
Para la carretera M-40 
Indica que hubo un atasco si la incidencia tiene hora fin 
Indica que existe un atasco si la incidencia no tiene hora fin 
'''

for x in diccionario:
    if "M-40" in x: 
        valores = diccionario.get(x) # obtener el valor correspondiente para el identificador 
        fin = "" # declarar variable para mensaje en caso de que la incidencia tenga hora fin
        cleantext =  cleanhtml(valores[5]) # quitar las etiquetas de html del texto que describe la incidencia 
        if valores[2] != '':  # En caso de que exista una hora fin
            mensaje = "Hubo un atasco en la carretera " + valores[4]
            fin = " el cual terminó a las " + valores[2] + " el día " + valores[3]
        else: 
            mensaje = "Existe un atasco en la carretera " + valores[4]
        mensaje = mensaje + " que inicio a las " + valores[0] + " el día " + valores[1]
        mensaje = mensaje + fin
        mensaje = mensaje + " con la siguientes características:\n" +  cleantext

        print(mensaje+"\n")
    


Existe un atasco en la carretera M-40 que inicio a las 17:14 el día 07/03/2019 con la siguientes características:
RETENCIÓN / CONGESTIÓN por ACCIDENTE y CIRCULACIÓN con circulación irregular  en:    - La AUTOPISTA / AUTOVÍA  M-40   a la altura de MADRID (MADRID) desde el  km 14 al  km 18 sentido DECRECIENTE DE LA KILOMETRACIÓN

Existe un atasco en la carretera M-40 que inicio a las 17:05 el día 07/03/2019 con la siguientes características:
OBSTÁCULO FIJO en:    -  La carretera   M-40   en el  km 14.815 en  (MADRID) sentido OESTE

Existe un atasco en la carretera M-40 que inicio a las 16:51 el día 07/03/2019 con la siguientes características:
RETENCIÓN / CONGESTIÓN por CIRCULACIÓN con circulación irregular  en:    - La AUTOPISTA / AUTOVÍA  M-40   a la altura de MADRID (MADRID) desde el  km 12 al  km 10 sentido CRECIENTE DE LA KILOMETRACIÓN



In [12]:
'''
Información extraída 
'''
# formato: {identificador: (horaInicio,fechaInicio, horaFin,fechaFin,carretera,descripcion), ...}
# el identificador está formado por la conctenación de la horaInicio, la fecha de inicio y la carretera
pprint(diccionario) #diccionario con datos de las incidencias

{'00:0001/01/2019CA-36': ('00:00',
                          '01/01/2019',
                          '23:59',
                          '31/12/2019',
                          'CA-36',
                          'RESTRICCIONES A MERCANCÍAS PELIGROSAS</b> por '
                          '<b>CIRCULACIÓN</b> en<b> Puente José León de '
                          'Carranza</b>: </span> <span style="margin-top:10px; '
                          'float:left; clear:both">  -  La carretera  <b> '
                          '<span style="color:#ab3000">CA-36 </span> </b> a la '
                          'altura de <b>CADIZ (CÁDIZ)</b> desde el <b> km '
                          '2.3</b> al <b> km 6</b> sentido <b>AMBOS SENTIDOS'),
 '02:0128/10/2018A-4025': ('02:01',
                           '28/10/2018',
                           '',
                           '',
                           'A-4025',
                           'CARRETERA CERRADA DE FORMA PERMANENTE POR '
                        